In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from tensorflow.keras.datasets import mnist
from sklearn.metrics import roc_auc_score
from tqdm.notebook import tqdm_notebook

from util import load_data, create_dataset

import time

In [2]:
(x_train, y_train), (x_test, y_test) = load_data(reshape=2)

In [3]:
def create_model():
    # CONV AE
    model = keras.Sequential([
        layers.Input( shape=(28, 28, 1) ),
        # Reduce ->
        layers.Conv2D(16, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D((2,2), padding='same'),
        # Reduce ->
        layers.Conv2D(8, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D((2,2), padding='same'),
        # Reduce -> 
        layers.Conv2D(8, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D((2,2), padding='same'),
        # Increase ->
        layers.Conv2D(8, (3,3), activation='relu', padding='same'),
        layers.UpSampling2D((2,2)),
        # Increase ->
        layers.Conv2D(8, (3,3), activation='relu', padding='same'),
        layers.UpSampling2D((2,2)),
        # Increase ->
        layers.Conv2D(16, (3,3), activation='relu'),
        layers.UpSampling2D((2,2)),
        # Final
        layers.Conv2D(1, (3,3), activation='sigmoid', padding='same')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy')
    return model

def fit_model(model, x_train, y_train, x_test, y_test, normal=4, verbose=1):
    callbacks = [
        keras.callbacks.EarlyStopping(patience=2)
    ]
    train = x_train[y_train==normal]
    test  = x_test[y_test==normal]
    history = model.fit(train, train,
                    epochs=50,
                    batch_size=256,
                    verbose=verbose,
                    shuffle=True,
                    validation_data=(test, test),
                    callbacks=callbacks)
    return model

In [4]:
model = create_model()
start = time.process_time()

model = fit_model(model, x_train, y_train, x_test, y_test, normal=4, verbose=0)

end = time.process_time()
print(end - start)

start = time.process_time()
xhat = model.predict(x_test)
end = time.process_time()
print(end - start)

914.796875
11.375


In [4]:
evals = np.zeros((10, 30))
for i in range(10):
    # Evaluate for all numbers
    for j in tqdm_notebook(range(5)):
        # Evaluate each method 30 times
        model = create_model()
        model = fit_model(model, x_train, y_train, x_test, y_test, normal=i)

        x = np.copy( x_test )
        y = y_test
        labels = np.copy( y )
        labels[ y == i ] = 0
        labels[ y != i ] = 1
        
        xhat = model.predict(x)
        
        x = x.reshape(len(x), 28*28)
        xhat = xhat.reshape(len(xhat), 28*28)
        
        err  = np.sum(np.abs(x-xhat), axis=1)
        # Max-Min normalize the error
        err /= np.max(err)
        # Compute AUC
        AUC = roc_auc_score(labels, err)
        evals[i,j] = AUC
    print(np.mean(evals[i,:])*100)

print(np.mean(evals[:,:5], axis=1))
print(np.std(evals, axis=1))

Epoch 1/50
24/24 [==============================] - 3s 137ms/step - loss: 0.5779 - val_loss: 0.5145
Epoch 2/50
24/24 [==============================] - 3s 133ms/step - loss: 0.4624 - val_loss: 0.3749
Epoch 3/50
24/24 [==============================] - 3s 128ms/step - loss: 0.3114 - val_loss: 0.2705
Epoch 4/50
24/24 [==============================] - 3s 123ms/step - loss: 0.2499 - val_loss: 0.2267
Epoch 5/50
24/24 [==============================] - 3s 128ms/step - loss: 0.2148 - val_loss: 0.2033
Epoch 6/50
24/24 [==============================] - 3s 123ms/step - loss: 0.1953 - val_loss: 0.1871
Epoch 7/50
24/24 [==============================] - 3s 122ms/step - loss: 0.1822 - val_loss: 0.1762
Epoch 8/50
24/24 [==============================] - 3s 124ms/step - loss: 0.1716 - val_loss: 0.1654
Epoch 9/50
24/24 [==============================] - 3s 124ms/step - loss: 0.1622 - val_loss: 0.1579
Epoch 10/50
24/24 [==============================] - 3s 129ms/step - loss: 0.1544 - val_loss: 0.1511

Epoch 1/50
27/27 [==============================] - 3s 118ms/step - loss: 0.6193 - val_loss: 0.3727
Epoch 2/50
27/27 [==============================] - 4s 132ms/step - loss: 0.3331 - val_loss: 0.2807
Epoch 3/50
27/27 [==============================] - 4s 133ms/step - loss: 0.2261 - val_loss: 0.1737
Epoch 4/50
27/27 [==============================] - 3s 129ms/step - loss: 0.1475 - val_loss: 0.1150
Epoch 5/50
27/27 [==============================] - 3s 126ms/step - loss: 0.1015 - val_loss: 0.0876
Epoch 6/50
27/27 [==============================] - 3s 126ms/step - loss: 0.0845 - val_loss: 0.0766
Epoch 7/50
27/27 [==============================] - 3s 121ms/step - loss: 0.0754 - val_loss: 0.0685
Epoch 8/50
27/27 [==============================] - 3s 118ms/step - loss: 0.0684 - val_loss: 0.0626
Epoch 9/50
27/27 [==============================] - 3s 116ms/step - loss: 0.0642 - val_loss: 0.0596
Epoch 10/50
27/27 [==============================] - 3s 115ms/step - loss: 0.0613 - val_loss: 0.0574

Epoch 1/50
24/24 [==============================] - 3s 122ms/step - loss: 0.5938 - val_loss: 0.4971
Epoch 2/50
24/24 [==============================] - 3s 133ms/step - loss: 0.4491 - val_loss: 0.3735
Epoch 3/50
24/24 [==============================] - 3s 129ms/step - loss: 0.3182 - val_loss: 0.2752
Epoch 4/50
24/24 [==============================] - 3s 127ms/step - loss: 0.2536 - val_loss: 0.2350
Epoch 5/50
24/24 [==============================] - 3s 123ms/step - loss: 0.2242 - val_loss: 0.2139
Epoch 6/50
24/24 [==============================] - 3s 121ms/step - loss: 0.2083 - val_loss: 0.2020
Epoch 7/50
24/24 [==============================] - 3s 120ms/step - loss: 0.1974 - val_loss: 0.1915
Epoch 8/50
24/24 [==============================] - 3s 117ms/step - loss: 0.1871 - val_loss: 0.1818
Epoch 9/50
24/24 [==============================] - 3s 116ms/step - loss: 0.1793 - val_loss: 0.1752
Epoch 10/50
24/24 [==============================] - 3s 115ms/step - loss: 0.1732 - val_loss: 0.1694

Epoch 1/50
24/24 [==============================] - 3s 131ms/step - loss: 0.5695 - val_loss: 0.4664
Epoch 2/50
24/24 [==============================] - 3s 132ms/step - loss: 0.4070 - val_loss: 0.3283
Epoch 3/50
24/24 [==============================] - 3s 131ms/step - loss: 0.2838 - val_loss: 0.2457
Epoch 4/50
24/24 [==============================] - 3s 135ms/step - loss: 0.2316 - val_loss: 0.2152
Epoch 5/50
24/24 [==============================] - 3s 125ms/step - loss: 0.2109 - val_loss: 0.1998
Epoch 6/50
24/24 [==============================] - 3s 125ms/step - loss: 0.1973 - val_loss: 0.1879
Epoch 7/50
24/24 [==============================] - 3s 121ms/step - loss: 0.1870 - val_loss: 0.1785
Epoch 8/50
24/24 [==============================] - 3s 119ms/step - loss: 0.1779 - val_loss: 0.1705
Epoch 9/50
24/24 [==============================] - 3s 121ms/step - loss: 0.1706 - val_loss: 0.1638
Epoch 10/50
24/24 [==============================] - 3s 118ms/step - loss: 0.1651 - val_loss: 0.1587

Epoch 1/50
23/23 [==============================] - 3s 130ms/step - loss: 0.5645 - val_loss: 0.4406
Epoch 2/50
23/23 [==============================] - 3s 125ms/step - loss: 0.3652 - val_loss: 0.2984
Epoch 3/50
23/23 [==============================] - 3s 126ms/step - loss: 0.2602 - val_loss: 0.2301
Epoch 4/50
23/23 [==============================] - 3s 125ms/step - loss: 0.2138 - val_loss: 0.1968
Epoch 5/50
23/23 [==============================] - 3s 123ms/step - loss: 0.1900 - val_loss: 0.1818
Epoch 6/50
23/23 [==============================] - 3s 122ms/step - loss: 0.1796 - val_loss: 0.1734
Epoch 7/50
23/23 [==============================] - 3s 125ms/step - loss: 0.1722 - val_loss: 0.1666
Epoch 8/50
23/23 [==============================] - 3s 128ms/step - loss: 0.1654 - val_loss: 0.1597
Epoch 9/50
23/23 [==============================] - 3s 126ms/step - loss: 0.1584 - val_loss: 0.1532
Epoch 10/50
23/23 [==============================] - 3s 124ms/step - loss: 0.1525 - val_loss: 0.1484

Epoch 1/50
22/22 [==============================] - 3s 124ms/step - loss: 0.5494 - val_loss: 0.4484
Epoch 2/50
22/22 [==============================] - 3s 130ms/step - loss: 0.4083 - val_loss: 0.3559
Epoch 3/50
22/22 [==============================] - 3s 137ms/step - loss: 0.2965 - val_loss: 0.2539
Epoch 4/50
22/22 [==============================] - 3s 137ms/step - loss: 0.2391 - val_loss: 0.2288
Epoch 5/50
22/22 [==============================] - 3s 131ms/step - loss: 0.2229 - val_loss: 0.2166
Epoch 6/50
22/22 [==============================] - 3s 135ms/step - loss: 0.2110 - val_loss: 0.2040
Epoch 7/50
22/22 [==============================] - 3s 126ms/step - loss: 0.1983 - val_loss: 0.1911
Epoch 8/50
22/22 [==============================] - 3s 133ms/step - loss: 0.1862 - val_loss: 0.1811
Epoch 9/50
22/22 [==============================] - 3s 123ms/step - loss: 0.1775 - val_loss: 0.1731
Epoch 10/50
22/22 [==============================] - 3s 125ms/step - loss: 0.1703 - val_loss: 0.1679

Epoch 1/50
24/24 [==============================] - 3s 125ms/step - loss: 0.6518 - val_loss: 0.5157
Epoch 2/50
24/24 [==============================] - 3s 123ms/step - loss: 0.4845 - val_loss: 0.4530
Epoch 3/50
24/24 [==============================] - 3s 122ms/step - loss: 0.3983 - val_loss: 0.3564
Epoch 4/50
24/24 [==============================] - 3s 121ms/step - loss: 0.3073 - val_loss: 0.2731
Epoch 5/50
24/24 [==============================] - 3s 120ms/step - loss: 0.2355 - val_loss: 0.2101
Epoch 6/50
24/24 [==============================] - 3s 121ms/step - loss: 0.1877 - val_loss: 0.1774
Epoch 7/50
24/24 [==============================] - 3s 120ms/step - loss: 0.1685 - val_loss: 0.1658
Epoch 8/50
24/24 [==============================] - 3s 124ms/step - loss: 0.1589 - val_loss: 0.1574
Epoch 9/50
24/24 [==============================] - 3s 120ms/step - loss: 0.1514 - val_loss: 0.1507
Epoch 10/50
24/24 [==============================] - 3s 120ms/step - loss: 0.1456 - val_loss: 0.1468

Epoch 1/50
25/25 [==============================] - 4s 150ms/step - loss: 0.5731 - val_loss: 0.4459
Epoch 2/50
25/25 [==============================] - 4s 148ms/step - loss: 0.3709 - val_loss: 0.3019
Epoch 3/50
25/25 [==============================] - 4s 164ms/step - loss: 0.2572 - val_loss: 0.2211
Epoch 4/50
25/25 [==============================] - 4s 157ms/step - loss: 0.2019 - val_loss: 0.1754
Epoch 5/50
25/25 [==============================] - 4s 156ms/step - loss: 0.1615 - val_loss: 0.1485
Epoch 6/50
25/25 [==============================] - 3s 137ms/step - loss: 0.1443 - val_loss: 0.1377
Epoch 7/50
25/25 [==============================] - 4s 148ms/step - loss: 0.1348 - val_loss: 0.1300
Epoch 8/50
25/25 [==============================] - 3s 131ms/step - loss: 0.1282 - val_loss: 0.1240
Epoch 9/50
25/25 [==============================] - 3s 130ms/step - loss: 0.1233 - val_loss: 0.1198
Epoch 10/50
25/25 [==============================] - 3s 129ms/step - loss: 0.1195 - val_loss: 0.1166

Epoch 1/50
23/23 [==============================] - 3s 145ms/step - loss: 0.6670 - val_loss: 0.5521
Epoch 2/50
23/23 [==============================] - 3s 128ms/step - loss: 0.4865 - val_loss: 0.4369
Epoch 3/50
23/23 [==============================] - 3s 135ms/step - loss: 0.3705 - val_loss: 0.3135
Epoch 4/50
23/23 [==============================] - 3s 138ms/step - loss: 0.2774 - val_loss: 0.2513
Epoch 5/50
23/23 [==============================] - 3s 134ms/step - loss: 0.2380 - val_loss: 0.2284
Epoch 6/50
23/23 [==============================] - 3s 134ms/step - loss: 0.2195 - val_loss: 0.2128
Epoch 7/50
23/23 [==============================] - 3s 131ms/step - loss: 0.2080 - val_loss: 0.2049
Epoch 8/50
23/23 [==============================] - 3s 130ms/step - loss: 0.2014 - val_loss: 0.1989
Epoch 9/50
23/23 [==============================] - 3s 139ms/step - loss: 0.1962 - val_loss: 0.1945
Epoch 10/50
23/23 [==============================] - 3s 134ms/step - loss: 0.1920 - val_loss: 0.1904

Epoch 1/50
24/24 [==============================] - 3s 142ms/step - loss: 0.6003 - val_loss: 0.4947
Epoch 2/50
24/24 [==============================] - 3s 140ms/step - loss: 0.4208 - val_loss: 0.3602
Epoch 3/50
24/24 [==============================] - 3s 143ms/step - loss: 0.2945 - val_loss: 0.2364
Epoch 4/50
24/24 [==============================] - 3s 133ms/step - loss: 0.2146 - val_loss: 0.1994
Epoch 5/50
24/24 [==============================] - 3s 132ms/step - loss: 0.1910 - val_loss: 0.1798
Epoch 6/50
24/24 [==============================] - 3s 134ms/step - loss: 0.1730 - val_loss: 0.1626
Epoch 7/50
24/24 [==============================] - 3s 139ms/step - loss: 0.1583 - val_loss: 0.1516
Epoch 8/50
24/24 [==============================] - 3s 138ms/step - loss: 0.1489 - val_loss: 0.1441
Epoch 9/50
24/24 [==============================] - 3s 137ms/step - loss: 0.1430 - val_loss: 0.1392
Epoch 10/50
24/24 [==============================] - 3s 138ms/step - loss: 0.1387 - val_loss: 0.1349

In [10]:
print(np.mean(evals[:,:5], axis=1))
print(np.std(evals[:,:5], axis=1))

[0.94176673 0.99850511 0.78951162 0.86800852 0.85905721 0.84941651
 0.88275971 0.91295657 0.78931156 0.90022523]
[0.03176284 0.0001717  0.01661758 0.01490975 0.00775537 0.03176802
 0.03015737 0.00639092 0.01094556 0.02750406]
